# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [9]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [10]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "proceedings": {
        "file" : "Proceedings.bib",
        "venuekey": "booktitle",
        "venue-pretext": "",
        "collection" : {"name":"Proceedings",
                        "permalink":"/proceeding/"}
        
    },
    "publications":{
        "file": "Publications.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"Publications",
                        "permalink":"/publication/"}
    },
    "preprints":{
        "file": "Preprints.bib",
        "venuekey" : None,
        "venue-pretext" : "",
        "collection" : {"name":"Preprints",
                        "permalink":"/preprint/"}
    }     
}

In [11]:
self_last_name = 'Beentjes'

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [13]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
                
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""
            
            citation_authors = ""
            #citation authors - todo - add highlighting for primary author?
            for author in bibdata.entries[bib_id].persons["author"]:
                if self_last_name in author.last_names[0]:
                    citation_authors += " <b>"+author.first_names[0]+" "+author.last_names[0]+"</b>, "                    
                else:
                    citation_authors += " "+author.first_names[0]+" "+author.last_names[0]+", "
            
            citation += citation_authors

            #citation title
            citation += "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""

            #add venue logic depending on citation type
            venue = ""
            if publist[pubsource]["venuekey"]:
                venue += publist[pubsource]["venue-pretext"] + \
                    b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")
                
                citation += " " + html_escape(venue) 
                try:
                    citation += " " + b["volume"]
                    if "number" in b:
                        citation += " (" + b["number"] + ")"
                    citation += ", " + b["pages"].replace("--","-")
                except:
                        pass
                citation += " (" + pub_year + ")."

            
            ## YAML variables
            md = "---\ntitle: '"  + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + \
                "'\n"
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True
            else:
                md += "\nexcerpt: ''"

            md += "\ndate: " + str(pub_date) 

            if publist[pubsource]["venuekey"]:
                md += "\nvenue: '" + html_escape(venue) + "'"
            
            md += "\npaper_authors: '" + html_escape(citation_authors[1:-2]) + "'"
            
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\npaperurl: '" + b["url"] + "'"
                    url = True
                    
            arXiv = False
            if "arxivId" in b.keys():
                if len(str(b["arxivId"])) > 5:
                    md += "\npreprinturl: '" + b["arxivId"] + "'"
                    arXiv = True                    

            md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"          
            
            ## Markdown description for individual page
            if note:
                md += "\n" + html_escape(b["note"]) + "\n"
                
            if "abstract" in b.keys():
                md += "\n" + "Abstract:\n" + html_escape(b["abstract"].replace("$\backslash$","\"")) + "\n"
                
#             if url:
#                 md += "\n[Access paper here](" + b["url"] + "){:target=\"_blank\"}\n" 
#             else:
#                 md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"} for full citation"

            md_filename = os.path.basename(md_filename)
            
            try:
                with open(f"../_{pubsource}/" + md_filename, 'x') as f:
                    f.write(md)
            except FileExistsError:
                pass
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


SUCESSFULLY PARSED Johnston2020: " {Accurate particle-based reaction algorithms for fixed times ... "
SUCESSFULLY PARSED Beentjes2017: " {Equalizing the cost of health insurance}  "
SUCESSFULLY PARSED Perez-Carrasco2020a: " {Effects of cell cycle variability on lineage and population ... "
SUCESSFULLY PARSED Beentjes2020: " {Exact solution of stochastic gene expression models with bu ... "
SUCESSFULLY PARSED Beentjes2019b: " {Defining vitamin D status using multi-metabolite mathematic ... "
SUCESSFULLY PARSED Beentjes2019: " {Uniformization techniques for stochastic simulation of chem ... "
SUCESSFULLY PARSED Beentjes2018: " {Quasi-Monte Carlo methods applied to tau-leaping in stochas ... "
SUCESSFULLY PARSED Farrell2016: " {The computation of disconnected bifurcation diagrams}  "
